In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from tqdm.notebook import tqdm
tqdm.pandas()

In [62]:
data_dir = './kaggle/input'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [63]:
df_train

,Id,GuideSeq,Fraction_Insertions,Avg_Deletion_Length,Indel_Diversity,Fraction_Frameshifts
0,0,CTGCAGGGCTAGTTTCCTATAGG,0.069572,4.301844,3.536538,0.807375
1,1,GAGATGCGGACCACCCAGCTGGG,0.287647,10.814444,3.871165,0.665696
2,2,GCAAACGGAAGTGCAATTGTCGG,0.137004,9.888889,3.931298,0.684823
3,3,GTCATCGCTGAGTTGAGGAAGGG,0.093889,4.527812,3.523067,0.753003
4,4,ATATGATTATCCCTGCACAAGGG,0.526525,6.415644,2.828101,0.887214
...,...,...,...,...,...,...
1060,1060,ATGCCCGACCAAAGACAACCAGG,0.093236,3.641787,3.170302,0.684801
1061,1061,CACGCTGTCATCCACCAGGTAGG,0.069054,7.661831,4.284711,0.639433
1062,1062,GGGCTCCAGAGTCTGATACAGGG,0.421439,7.364856,3.175859,0.832975
1063,1063,AAAGACTTCGGTCCTCTAGTAGG,0.206277,7.907886,3.782588,0.528375


In [64]:
cat = pd.CategoricalDtype(categories=['A', 'T', 'C', 'G'])

letters = ['A', 'T', 'G', 'C']
max_length = 1


def add_cols(data: pd.DataFrame):
    seq_len = len(data['GuideSeq'].iloc[0])
    
    acc = data
    for l in tqdm(range(1, max_length+1)):
        for pos in tqdm(range(0, seq_len - l + 1)):
            indices = ':'.join(str(i) for i in range(pos, l+pos))
            col_name = f'pos_{l}_{indices}'
            col_data = data["GuideSeq"].str.slice(start=pos, stop=pos+l)
            acc = pd.concat([acc, col_data.rename(col_name).astype('category')], axis=1)
            # for combination in combinations:
            #     col_name_2 = col_name + "_is_" + combination
            #     col_data_2 = col_data == combination
            #     acc = pd.concat([acc, col_data_2.rename(col_name_2)], axis=1)
    return acc

# x = make_letters_cols(df_train)

# def add_cols(df: pd.DataFrame):
#     split_columns = df['GuideSeq'].apply(lambda x: pd.Series(list(x)))
#     split_columns.columns = [f'pos_{i+1}' for i in range(split_columns.shape[1])]
#     exp = pd.concat([df, split_columns], axis=1)
#     for col in exp.columns:
#         if col.startswith('pos_'):
#             exp[col] = exp[col].astype(cat)
#     return exp

data_expanded = add_cols(df_train)
test_data_expanded = add_cols(df_test)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

In [98]:
data_expanded.filter(regex='^pos_1')

,pos_1_0,pos_1_1,pos_1_2,pos_1_3,pos_1_4,pos_1_5,pos_1_6,pos_1_7,pos_1_8,pos_1_9,...,pos_1_13,pos_1_14,pos_1_15,pos_1_16,pos_1_17,pos_1_18,pos_1_19,pos_1_20,pos_1_21,pos_1_22
0,C,T,G,C,A,G,G,G,C,T,...,T,T,C,C,T,A,T,A,G,G
1,G,A,G,A,T,G,C,G,G,A,...,C,C,C,A,G,C,T,G,G,G
2,G,C,A,A,A,C,G,G,A,A,...,C,A,A,T,T,G,T,C,G,G
3,G,T,C,A,T,C,G,C,T,G,...,T,G,A,G,G,A,A,G,G,G
4,A,T,A,T,G,A,T,T,A,T,...,T,G,C,A,C,A,A,G,G,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,A,T,G,C,C,C,G,A,C,C,...,G,A,C,A,A,C,C,A,G,G
1061,C,A,C,G,C,T,G,T,C,A,...,A,C,C,A,G,G,T,A,G,G
1062,G,G,G,C,T,C,C,A,G,A,...,T,G,A,T,A,C,A,G,G,G
1063,A,A,A,G,A,C,T,T,C,G,...,C,T,C,T,A,G,T,A,G,G


In [133]:

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint

epsilon = 0.01
def get_model(X, y, std):
    
    X_train = X
    y_train = y
    std_train = std
    # X_train, X_test, y_train, y_test, std_train, std_test = train_test_split(X, y, std, test_size=0.01, random_state=42)

    param_distributions = {
        'n_estimators': [100, 300, 600, 1000],
        'max_depth': [4, 10],
        'learning_rate': [0.03],
        'subsample': uniform(0, 1),
        'reg_alpha': uniform(0, 15.0),
        'reg_lambda': uniform(0, 15.0)
    }
    # sample_weights = 1 / (std_train + epsilon)

    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    random_search = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=param_distributions,
        n_iter=500,
        cv=kf,
        random_state=42,
        n_jobs=-1,
        error_score="raise"
    )

    random_search.fit(X_train, y_train, sample_weight=std_train)

    best_model = random_search.best_estimator_
    # best_model = xgb_model
    best_model.fit(X_train, y_train, sample_weight=std_train)

    y_pred = best_model.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    print(f"Test Mean Squared Error: {mse}")
    return best_model

features = ['Fraction_Insertions', 'Avg_Deletion_Length', 'Fraction_Frameshifts', 'Indel_Diversity']

models = {}

# models['Indel_Diversity'] = get_model(data_expanded.filter(regex='^pos_1'), data_expanded['Indel_Diversity'])

for feature in features:
    models[feature] = get_model(data_expanded.filter(regex='^pos_1'), data_expanded[feature], data_expanded['Indel_Diversity'])


/Users/timurseitosmanov/repos/mlfls-hw/.pixi/envs/default/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Test Mean Squared Error: 0.0031597233723964237


/Users/timurseitosmanov/repos/mlfls-hw/.pixi/envs/default/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Test Mean Squared Error: 5.657916499395398


/Users/timurseitosmanov/repos/mlfls-hw/.pixi/envs/default/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Test Mean Squared Error: 0.00915390084707154


/Users/timurseitosmanov/repos/mlfls-hw/.pixi/envs/default/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Test Mean Squared Error: 0.13829411057171495


In [130]:
models_500_n_iter_no_std = models

In [17]:
features = ['Fraction_Insertions', 'Avg_Deletion_Length', 'Indel_Diversity', 'Fraction_Frameshifts']
X_train = data_expanded.filter(regex='^pos')
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
                         'max_depth': 5, 'alpha': 10}
xgb_cv = xgb.cv(dtrain=data_expanded, nfold=5, params=params,
                        num_boost_round=50, early_stopping_rounds=10, metrics=features, as_pandas=True, seed=123)
# for feature in features:
#     y_train = data_expanded[feature]

#     params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
#                         'max_depth': 5, 'alpha': 10}



AttributeError: 'DataFrame' object has no attribute 'get_uint_info'

In [134]:
x = pd.DataFrame([{'name': feature, **models[feature].get_params()} for feature in features])
x

,name,objective,base_score,booster,callbacks,colsample_bylevel,colsample_bynode,colsample_bytree,device,early_stopping_rounds,...,num_parallel_tree,random_state,reg_alpha,reg_lambda,sampling_method,scale_pos_weight,subsample,tree_method,validate_parameters,verbosity
0,Fraction_Insertions,reg:squarederror,None,None,None,None,None,None,None,None,...,None,None,1.369300,4.789705,None,None,0.950062,None,None,None
1,Avg_Deletion_Length,reg:squarederror,None,None,None,None,None,None,None,None,...,None,None,7.030397,6.222293,None,None,0.273407,None,None,None
2,Fraction_Frameshifts,reg:squarederror,None,None,None,None,None,None,None,None,...,None,None,3.659845,14.595158,None,None,0.393098,None,None,None
3,Indel_Diversity,reg:squarederror,None,None,None,None,None,None,None,None,...,None,None,2.339918,0.871254,None,None,0.866176,None,None,None


In [135]:

t = test_data_expanded.filter(regex='^pos_|Id').copy()

t.insert(0, 'Indel_Diversity', models['Indel_Diversity'].predict(t.filter(regex='^pos_')))
t['Fraction_Insertions'] = models['Fraction_Insertions'].predict(t.filter(regex='^pos_'))
t['Avg_Deletion_Length'] = models['Avg_Deletion_Length'].predict(t.filter(regex='^pos_'))
t['Fraction_Frameshifts'] = models['Fraction_Frameshifts'].predict(t.filter(regex='^pos_'))
t[['Id', 'Fraction_Insertions', 'Avg_Deletion_Length', 'Indel_Diversity', 'Fraction_Frameshifts']].to_csv('./kaggle/output/out.csv', index=False)
t



,Indel_Diversity,Id,pos_1_0,pos_1_1,pos_1_2,pos_1_3,pos_1_4,pos_1_5,pos_1_6,pos_1_7,...,pos_1_16,pos_1_17,pos_1_18,pos_1_19,pos_1_20,pos_1_21,pos_1_22,Fraction_Insertions,Avg_Deletion_Length,Fraction_Frameshifts
0,4.105932,0,T,G,T,G,C,A,A,T,...,C,T,A,A,G,G,G,0.170639,7.061314,0.698551
1,3.234406,1,T,G,T,C,T,G,G,C,...,T,A,C,A,G,G,G,0.333440,6.362437,0.781285
2,3.612014,2,A,C,T,G,A,G,A,G,...,A,A,A,G,T,G,G,0.205031,6.829690,0.690110
3,3.820130,3,G,T,T,C,T,G,C,A,...,T,T,C,A,C,G,G,0.286609,6.989613,0.761625
4,4.041420,4,A,C,T,G,G,A,T,G,...,T,G,G,T,G,G,G,0.182061,8.391965,0.638358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,4.011418,299,T,A,T,G,A,A,G,C,...,G,A,G,T,C,G,G,0.098339,6.817652,0.680105
300,3.842157,300,A,T,G,T,T,G,G,A,...,C,G,G,T,G,G,G,0.216522,8.046727,0.676529
301,3.730892,301,G,A,C,C,A,G,G,G,...,T,T,A,A,A,G,G,0.176996,7.687791,0.712689
302,4.387889,302,A,G,T,A,G,A,C,T,...,A,G,G,A,G,G,G,0.169891,8.837729,0.697237
